# Notebook to compare configs of device(s)
___To use this notebook...___
1. configure your username and password
2. set backup_dir ir get_config_from_device to get configs from devices
3. set where statement to get the list of devices
4. configure token to access the sot
5. If you want to see some more DEBUG messages set loglevel to DEBUG (optional)

### define your login and set backup Directory

In [ ]:
username = "username"
password = "password"

### use backup_dir or get configs from device

In [ ]:
get_config_from_device=True
ssh_port=22
backup_dir = "/Users/marc/Programming/network-automation/device_configs"

### on which devices should the configs be compared

In [ ]:
where = "name=lab.local"

### configure our SOT

In [ ]:
token = "your_token"
url = "http://127.0.0.1:8080"
ssl_verify = False

### set loglevel

In [ ]:
loglevel = "CRITICAL"
scrapli_loglevel = "NONE"

# Basic setup .. import, logging and so on

In [ ]:
import pandas as pd
import sys
import difflib
import glob
import os
from loguru import logger
from veritas.sot import sot as veritas_sot
from veritas.devicemanagement import devicemanagement as dm
from IPython.display import display, HTML

In [ ]:
logger.configure(extra={"extra": "unset"})
logger.remove()
logger.add(sys.stderr, level=loglevel)

In [ ]:
def get_configs(hostname, directory, live=False, ip=None, 
                username=None, password=None, platform='ios', manufacturer='cisco', port=22, 
                scrapli_loglevel='none'):

    if live:
        conn = dm.Devicemanagement(ip=ip, platform=platform, manufacturer=manufacturer, username=username, password=password, port=port, scrapli_loglevel=scrapli_loglevel)
        startup_config = conn.get_config('startup-config').splitlines()
        running_config = conn.get_config('running-config').splitlines()
    else:
        running_filename = f'{directory}/{hostname}.running.cfg'
        startup_filename = f'{directory}/{hostname}.startup.cfg'
        with open(startup_filename, 'r') as sf:
            startup_config = [line.rstrip() for line in sf]
        with open(running_filename, 'r') as rf:
            running_config = [line.rstrip() for line in rf]

    return startup_config, running_config

## initialize our sot and get the list of devices

In [ ]:
sot = veritas_sot.Sot(token=token, url=url, ssl_verify=ssl_verify)

In [ ]:
devices = sot.select('name, primary_ip4, device_type, platform') \
             .using('nb.devices') \
             .where(where)

#### if you want to see the list of devices run the next cell

In [ ]:
# display(devices)

# see differences of devices

In [ ]:
for device in devices:
    hostname = device.get('name')
    ip = device.get('primary_ip4',{}).get('address').split('/')[0]
    platform = device.get('platform',{}).get('name')
    manufacturer = device.get('device_type',{}).get('manufacturer',{}).get('name','cisco')

    directory = f'{backup_dir}/{hostname}/'
    logger.bind(extra=hostname).debug(f'comparing configs')

    startup_config, running_config = get_configs(hostname, directory, get_config_from_device, ip, username, password, platform, manufacturer, ssh_port)
    html_diff = difflib.HtmlDiff()
    comparison_table = html_diff.make_file(startup_config, running_config)
    display(HTML(comparison_table))

### show hostname, software, hardware, serial, and uptime

In [ ]:
command = "show version"
version = conn.send(command)
df = pd.DataFrame(version[command])
df[['hostname', 'software_image', 'version', 'uptime', 'hardware', 'serial', 'config_register']]

### show cdp neighbors

In [ ]:
command = "show cdp neighbors"
neighbors = conn.send(command)
df = pd.DataFrame(neighbors[command])
df

### show OSPF neighbors

In [ ]:
command = "show ip ospf neighbors"
ospf_neighbors = conn.send(command)
df = pd.DataFrame(ospf_neighbors[command])
df

### show ip int brief

In [ ]:
command = "show ip int brief"
int_brief = conn.send(command)
df = pd.DataFrame(int_brief[command])
df

### SNMP user

In [ ]:
command = "show snmp user"
int_brief = conn.send(command)
df = pd.DataFrame(int_brief[command])
df

In [ ]:
command = "show ip route"
ip_route = conn.send(command)
df = pd.DataFrame(ip_route[command])
df

In [ ]:
command = "show ip arp"
arp_table = conn.send(command)
df = pd.DataFrame(arp_table[command])
df

## show interfaces

In [ ]:
command = "show interfaces"
arp_table = conn.send(command)
df = pd.DataFrame(arp_table[command])
df

## show ip interface brief

In [ ]:
command = "show ip interface brief"
int_brief = conn.send(command)
df = pd.DataFrame(int_brief[command])
df

## show interfaces description

In [ ]:
command = "show interfaces description"
arp_table = conn.send(command)
df = pd.DataFrame(arp_table[command])
df